In [1]:
import geopandas as gpd
import pandas as pd
from utils import gdf_operations, functions

2025-01-29 15:35:24.989 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
distrito =gdf_operations.get_dados('distrito')
subbac = gdf_operations.get_dados('subbac')
subpref = gdf_operations.get_dados('subpref')
fcu = gdf_operations.get_dados('fcu')

In [3]:
cd_subbac = gdf_operations.find_subbac_name(subbac, 'cd_')
nm_subbac = gdf_operations.find_subbac_name(subbac, 'nm_')

unidades_list = [
    ('subbac', "Sub Bacias Hidrográficas", cd_subbac, nm_subbac),
    ('subpref', "Subprefeituras", "cd_subpref" , 'nm_subpref'),
    ('distrito', "Distritos", 'cd_distrit', 'nm_distrit'),
    ('fcu', "Favelas e Comunidades Urbanas", 'cd_fcu', 'nm_fcu')
    
]
unidades_df = pd.DataFrame(unidades_list, columns=['gdf_name', 'name', 'column_cd', 'column_name'])


Vou começar pelo fcu para ver se dá certo.

# FCU

In [4]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [5]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
fcu
column_name


overlay_gdf = gdf_operations.overlay_intersec(
    'distrito', gdf_unidade, 'cd_fcu')

# Distritos

Agora vamos tentar com os distritos

In [6]:
distrito_sorted = gdf_operations.create_gdf_sorted(
    distrito, 'distrito'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [7]:
choice_unidade = 'distrito'
gdf_unidade = distrito_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
distrito
column_name


In [8]:
overlay_gdf = gdf_operations.overlay_intersec(
    'subpref', gdf_unidade, 'cd_distrit'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


# Loop por intersec

## Vamos Começar com o FCU mesmo

In [9]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [10]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
fcu
column_name


In [11]:
print(unidades_df[unidades_df['gdf_name']=='fcu'])

  gdf_name                           name column_cd column_name
3      fcu  Favelas e Comunidades Urbanas    cd_fcu      nm_fcu


In [ ]:
index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == choice_unidade
        ].index[0]
    )

name_gdf_unidade = choice_unidade
batatinha = pd.DataFrame()

cd_unidade = gdf_operations.find_gdf_name(gdf_unidade, choice_unidade, 'cd_')

overlay_gdf = gpd.GeoDataFrame()


for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )
            print(overlay_gdf.columns)
            
            gdf_unidade[cd_unidade] = gdf_unidade[cd_unidade].astype(str)
            overlay_gdf[cd_intersec] =(
                overlay_gdf[cd_intersec]
                .astype(int)
                .astype(str)
            )
            
            overlay_gdf[cd_unidade] = (
                overlay_gdf[cd_unidade]
                .astype(str)
            )


            gdf_unidade = pd.merge(gdf_unidade, overlay_gdf[[cd_unidade, cd_intersec]], on=cd_unidade, how='left')


gdf_operations.save_intersec(gdf_unidade, choice_unidade)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


In [ ]:
gdf_unidade.sample(3)

## Intersec distrito

In [ ]:
distrito_sorted = gdf_operations.create_gdf_sorted(
    distrito, 'distrito'
)
choice_unidade = 'distrito'
gdf_unidade = distrito_sorted
name_column_unidade = functions.find_gdf_info(
    unidades_df, 
    choice_unidade, 
    'gdf_name', 
    'column_name'
)

In [ ]:
index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == choice_unidade
        ].index[0]
    )

name_gdf_unidade = choice_unidade
batatinha = pd.DataFrame()

cd_unidade = gdf_operations.find_gdf_name(gdf_unidade, choice_unidade, 'cd_')

overlay_gdf = gpd.GeoDataFrame()


for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )
            print(overlay_gdf.columns)
            
            gdf_unidade[cd_unidade] = gdf_unidade[cd_unidade].astype(str)
            overlay_gdf[cd_intersec] = overlay_gdf[cd_intersec].astype(int).astype(str)
            overlay_gdf[cd_unidade] = overlay_gdf[cd_unidade].astype(str)


            gdf_unidade = pd.merge(gdf_unidade, overlay_gdf[[cd_unidade, cd_intersec]], on=cd_unidade, how='left')


gdf_operations.save_intersec(gdf_unidade, choice_unidade)


In [ ]:
gdf_unidade.sample(3)

## Testar DE NOVO O LOOP

In [ ]:
def intersec_unidades(
    unidades_df,
    gdf_unidade:gpd.GeoDataFrame,
    name_gdf_unidade:str
    
):
    gdf_sorted = gdf_operations.create_gdf_sorted(
        gdf_unidade, 
        name_gdf_unidade
    )

    gdf_unidade= gdf_sorted

    name_column_unidade = gdf_operations.find_gdf_name(
        gdf_unidade, name_gdf_unidade, 'nm_'
    )
#    name_column_unidade = functions.find_gdf_info(unidades_df, name_gdf_unidade, 'gdf_name', 'column_name')

    index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == name_gdf_unidade
        ].index[0]
    )

  

    cd_unidade = gdf_operations.find_gdf_name(
        gdf_unidade, 
        name_gdf_unidade, 
        'cd_'
    )

    overlay_gdf = gpd.GeoDataFrame()

    for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )

            
            gdf_unidade[cd_unidade] = (
                gdf_unidade[cd_unidade].astype(str)
            )
            overlay_gdf[cd_intersec] = (
                overlay_gdf[cd_intersec]
                .astype(int)
                .astype(str)
            )
            overlay_gdf[cd_unidade] = (
                overlay_gdf[cd_unidade].astype(str)
            )

    


            gdf_unidade = pd.merge(
                gdf_unidade, 
                overlay_gdf[[cd_unidade, cd_intersec]], 
                on=cd_unidade, 
                how='left'
            )

    
    gdf_operations.save_intersec(gdf_unidade, choice_unidade)

    return gdf_unidade
            

intersec_fcu = intersec_unidades(unidades_df, fcu, 'fcu')
intersec_fcu.sample(3)

In [ ]:
intersec_distrito = intersec_unidades(unidades_df, distrito, 'distrito')
intersec_distrito.sample(3)